# FC25 Rush Detection Model
## Deploy as a Service
This notebook will deploy the Rush detection model as a service. The goal is to leverage as much project code as possible, and to keep this notebook as lightweight as possible.

Ideally, this notebook will be run on a local subnet, and the service will be accessed via a local IP address. Alternatively, it could be run on colab, and accessed via a public address.

### Setup
From your colab notebook, you should be able to run the following commands to setup the environment. Don't forget to create a `.env` file in the `src` directory with your API keys.
`HF_API_KEY`
`HF_RUSH_DETECTION_PATH`
`HF_RUSH_DETECTION_FILENAME`
`NGROK_AUTH_TOKEN`
```
%%capture
%%sh
git clone https://github.com/swaynos/urban-palm-tree.git

%cd urban-palm-tree/src
```

In [ ]:
import sys
import os

# Check if the current working directory is under 'notebooks'
if 'notebooks' in os.getcwd().split(os.sep):
    # Adjust the path to include the 'src' directory
    sys.path.append(os.path.abspath(os.path.join('..', 'src')))  # This adds the correct src path

# Verify the current system path
print(sys.path)

## Run the Service from Jupyter

### Step 1. Environment Setup
Install the required packages.

In [ ]:
%%capture
%%sh
pip install aiofiles dotenv fastapi huggingface_hub python-multipart uvicorn pillow numpy gunicorn scikit-image torch ultralytics

### Step 2. 
The endpoint code.

### Step 3. Run the Service

In [ ]:
import multiprocessing
import time
import torch
from multiprocessing import Pool, Manager
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from PIL import Image
import io
import uvicorn
import nest_asyncio
from utilities.shared_thread_resources import SharedProgramData
from inference.rush_inference import run_inference

PORT=6969

if __name__ == "__main__":
    multiprocessing.set_start_method("spawn", force=True)

    # ✅ Apply nest_asyncio to allow Uvicorn to run in Jupyter Notebook
    nest_asyncio.apply()

    # ✅ Create a multiprocessing Manager and Queue
    manager = Manager()
    queue = manager.Queue()

    # ✅ Create a persistent worker pool (reuses processes instead of creating new ones every time)
    pool = Pool(processes=1)  # Adjust worker count based on system capabilities

    # ✅ Initialize FastAPI app
    app = FastAPI()

    @app.post("/detect")
    async def detect_objects(file: UploadFile = File(...)):
        """Handles image uploads and returns YOLO object detection results."""
        try:
            start_time = time.time()

            # Read image file into memory
            contents = await file.read()
            image = Image.open(io.BytesIO(contents)).convert('RGB')

            # ✅ Use the persistent worker pool and managed queue
            pool.apply(run_inference, args=(image, queue))

            # ✅ Get results from queue (prevent hanging)
            if not queue.empty():
                results = queue.get()
            else:
                results = {"error": "Queue was empty, worker did not return results"}

            end_time = time.time()
            print(f"[Server] Total request processing time: {end_time - start_time:.4f} sec")

            return JSONResponse(content=results, status_code=200)

        except Exception as e:
            print(f"[Server] ERROR: {e}")
            return JSONResponse(content={'error': str(e)}, status_code=500)

    # ✅ Run Uvicorn in a background thread
    import threading
    server_thread = threading.Thread(target=lambda: uvicorn.run(app, host="0.0.0.0", port=PORT), daemon=True)
    server_thread.start()


#### Running the Service from Google Colab

In [ ]:
# If running on Colab
!pip install pyngrok
from pyngrok import ngrok
import os

# Authenticate ngrok with your token
NGROK = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK)

# Set up ngrok tunnel
public_url = ngrok.connect(PORT)
print(f"ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{PORT}\"")
